In [1]:
from lib.datasets import ATPBind3D

# dataset_orig = ATPBind3D()
# dataset_orig.initialize_mask_and_weights()

In [7]:
dataset = ATPBind3D(to_slice=True)
dataset.initialize_mask_and_weights()

Initialize Undersampling: all ones
Initialize Weighting: all ones


ATPBind3D(
  #sample: 631
  #task: 1
)

In [11]:
from lib.pipeline import Pipeline
import pandas as pd
from torchdrug import utils, data
import os
import torch
import inspect
import numpy as np

from lib.utils import generate_mean_ensemble_metrics_auto, read_initial_csv, aggregate_pred_dataframe
GPU = 0

from atpbind_main import main


In [2]:
main(model_key='esm-t33', valid_fold=0)

batch_size: 8
init pipeline, model: esm-t33, dataset: atpbind3d, gpus: [0]
load model esm-t33, kwargs: {'gpu': 0, 'freeze_esm': False, 'freeze_layer_count': 30}
get dataset atpbind3d
Initialize Undersampling: all ones
Initialize Weighting: all ones
train samples: 302, valid samples: 76, test samples: 41
use cyclic lr scheduler
pipeline batch_size: 8
0m20s {'sensitivity': 0.5949, 'precision': 0.5902, 'mcc': 0.5702, 'micro_auprc': 0.6137, 'valid_mcc': 0.541, 'train_bce': 0.166, 'valid_bce': 0.1067, 'lr': 0.00084}
0m19s {'sensitivity': 0.6938, 'precision': 0.6601, 'mcc': 0.6586, 'micro_auprc': 0.7047, 'valid_mcc': 0.6246, 'train_bce': 0.0828, 'valid_bce': 0.0937, 'lr': 0.00138}
0m20s {'sensitivity': 0.6204, 'precision': 0.6946, 'mcc': 0.6389, 'micro_auprc': 0.6339, 'valid_mcc': 0.6264, 'train_bce': 0.0429, 'valid_bce': 0.1536, 'lr': 0.00192}
0m21s {'sensitivity': 0.6715, 'precision': 0.6651, 'mcc': 0.6501, 'micro_auprc': 0.6658, 'valid_mcc': 0.6298, 'train_bce': 0.0323, 'valid_bce': 0.124

In [1]:
from lib.pipeline import Pipeline
from torchdrug import utils, data
import torch
import numpy as np

CYCLE_SIZE = 10
pipeline = Pipeline(
    dataset='atpbind3d',
    model='lm-gearnet',
    gpus=[1],
    model_kwargs={
        'gpu': 1,
        'lm_type': 'esm-t33',
        'gearnet_hidden_dim_size': 512,
        'gearnet_hidden_dim_count': 4,
        'lm_freeze_layer_count': 30,
    },
    scheduler='cyclic',
    scheduler_kwargs={
        'base_lr': 3e-4,
        'max_lr': 3e-3,
        'step_size_up': CYCLE_SIZE / 2,
        'step_size_down': CYCLE_SIZE / 2,
        'cycle_momentum': False
    },
    batch_size=8,
    dataset_kwargs={
        'to_slice': True,
        'max_slice_length': 450,
        'padding': 50,
    },
    valid_fold_num=0,
)

init pipeline, model: lm-gearnet, dataset: atpbind3d, gpus: [1]
load model lm-gearnet, kwargs: {'gpu': 1, 'lm_type': 'esm-t33', 'gearnet_hidden_dim_size': 512, 'gearnet_hidden_dim_count': 4, 'lm_freeze_layer_count': 30}
freeze_lm: 30
get dataset with kwargs: {'to_slice': True, 'max_slice_length': 450, 'padding': 50}
get dataset atpbind3d
Initialize Undersampling: all ones
Initialize Weighting: all ones
train samples: 396, valid samples: 99, test samples: 41
use cyclic lr scheduler
pipeline batch_size: 8


In [2]:
pipeline.evaluate_test_sliced()

Protein(num_atom=2020, num_bond=4036, num_residue=505)


Protein(num_atom=1320, num_bond=2638, num_residue=330)
Protein(num_atom=1244, num_bond=2480, num_residue=311)
Protein(num_atom=656, num_bond=1308, num_residue=164)
Protein(num_atom=460, num_bond=918, num_residue=115)
Protein(num_atom=1260, num_bond=2514, num_residue=315)
Protein(num_atom=1700, num_bond=3394, num_residue=425)
Protein(num_atom=716, num_bond=1426, num_residue=179)
Protein(num_atom=944, num_bond=1886, num_residue=236)
Protein(num_atom=904, num_bond=1806, num_residue=226)
Protein(num_atom=908, num_bond=1812, num_residue=227)
Protein(num_atom=2120, num_bond=4238, num_residue=530)
Protein(num_atom=2656, num_bond=5310, num_residue=664)
Protein(num_atom=2424, num_bond=4846, num_residue=606)
Protein(num_atom=1116, num_bond=2224, num_residue=279)
Protein(num_atom=2520, num_bond=5038, num_residue=630)
Protein(num_atom=3452, num_bond=6896, num_residue=863)
Protein(num_atom=1592, num_bond=3180, num_residue=398)
Protein(num_atom=640, num_bond=1278, num_residue=160)
Protein(num_atom=1

{'sensitivity': 0.0,
 'precision': nan,
 'mcc': 0.0,
 'micro_auprc': 0.03160519897937775}

In [3]:
test_set = pipeline.test_set

In [5]:
from lib.pipeline import create_single_pred_dataframe

for protein_index, batch in enumerate(data.DataLoader(test_set, batch_size=1, shuffle=False)):
    print(batch)
    break


{'graph': PackedProtein(batch_size=1, num_atoms=[2020], num_bonds=[4036], num_residues=[505])}


In [14]:
protein = batch['graph'].unpack()[0]

In [18]:
batch = utils.cuda(batch, device=torch.device(f'cuda:{pipeline.gpus[0]}'))
pipeline.task.predict(batch)

tensor([[ -0.9557],
        [ -2.3748],
        [ -6.1451],
        [-18.1404],
        [-19.5787],
        [ -6.5335],
        [ -3.3242],
        [ -0.8382],
        [ -2.9643],
        [ -8.6353],
        [ -3.0043],
        [ -0.7857],
        [ -1.1304],
        [ -1.2272],
        [ -1.5017],
        [ -1.0366],
        [ -0.9504],
        [ -0.5819],
        [ -0.5903],
        [ -1.1333],
        [ -1.4306],
        [ -4.0878],
        [ -7.3961],
        [-10.1201],
        [-10.4953],
        [-13.8481],
        [-20.2002],
        [-27.5310],
        [-29.9036],
        [-33.3760],
        [-28.1787],
        [-25.1869],
        [-23.4030],
        [-14.3381],
        [-17.7067],
        [-24.2213],
        [-23.7300],
        [-34.6512],
        [-36.7245],
        [-37.3546],
        [-34.4552],
        [-25.2963],
        [-20.6385],
        [-16.4741],
        [ -8.5749],
        [-16.2357],
        [-15.8700],
        [-17.4560],
        [-21.4262],
        [-18.1280],


torch.Size([505, 1])

In [15]:
protein

Protein(num_atom=2020, num_bond=4036, num_residue=505)

In [2]:
pipeline.train_until_fit(patience=10, max_epoch=10)

evaluate test sliced with threshold: -0.3999999999999999, max_slice_length: 650, padding: 50
1m1s {'sensitivity': 0.436, 'precision': 0.6599, 'mcc': 0.5188, 'micro_auprc': 0.5559, 'valid_mcc': 0.512, 'train_bce': 0.1818, 'valid_bce': 0.1203, 'lr': 0.00084}
evaluate test sliced with threshold: -1.0999999999999999, max_slice_length: 650, padding: 50
0m59s {'sensitivity': 0.6295, 'precision': 0.5127, 'mcc': 0.5451, 'micro_auprc': 0.5876, 'valid_mcc': 0.6111, 'train_bce': 0.0756, 'valid_bce': 0.0855, 'lr': 0.00138}
evaluate test sliced with threshold: -0.09999999999999964, max_slice_length: 650, padding: 50
0m58s {'sensitivity': 0.6473, 'precision': 0.5746, 'mcc': 0.5899, 'micro_auprc': 0.6375, 'valid_mcc': 0.6133, 'train_bce': 0.0495, 'valid_bce': 0.0984, 'lr': 0.00192}
evaluate test sliced with threshold: -1.0, max_slice_length: 650, padding: 50
0m59s {'sensitivity': 0.5729, 'precision': 0.6975, 'mcc': 0.6163, 'micro_auprc': 0.6295, 'valid_mcc': 0.6264, 'train_bce': 0.0303, 'valid_bce': 

[{'sensitivity': 0.436,
  'precision': 0.6599,
  'mcc': 0.5188,
  'micro_auprc': 0.5559,
  'valid_mcc': 0.512,
  'train_bce': 0.1818,
  'valid_bce': 0.1203,
  'lr': 0.00084},
 {'sensitivity': 0.6295,
  'precision': 0.5127,
  'mcc': 0.5451,
  'micro_auprc': 0.5876,
  'valid_mcc': 0.6111,
  'train_bce': 0.0756,
  'valid_bce': 0.0855,
  'lr': 0.00138},
 {'sensitivity': 0.6473,
  'precision': 0.5746,
  'mcc': 0.5899,
  'micro_auprc': 0.6375,
  'valid_mcc': 0.6133,
  'train_bce': 0.0495,
  'valid_bce': 0.0984,
  'lr': 0.00192},
 {'sensitivity': 0.5729,
  'precision': 0.6975,
  'mcc': 0.6163,
  'micro_auprc': 0.6295,
  'valid_mcc': 0.6264,
  'train_bce': 0.0303,
  'valid_bce': 0.1206,
  'lr': 0.00246},
 {'sensitivity': 0.6131,
  'precision': 0.6398,
  'mcc': 0.6087,
  'micro_auprc': 0.6371,
  'valid_mcc': 0.6487,
  'train_bce': 0.0203,
  'valid_bce': 0.1023,
  'lr': 0.003},
 {'sensitivity': 0.5759,
  'precision': 0.6973,
  'mcc': 0.6178,
  'micro_auprc': 0.6213,
  'valid_mcc': 0.6257,
  'tra

In [3]:
from lib.datasets import get_slices, protein_to_slices
from lib.utils import dict_tensor_to_num, round_dict

dataset = pipeline.dataset

from torchdrug import transforms
pipeline.dataset.transform = transforms.ProteinView(view='residue')

train_set, valid_set, test_set = pipeline.dataset.split()

def infer_sliced(pipeline, protein, max_slice_length=650, padding=50):
    GPU = 1
    target = protein.target
    intermediate_preds = []
    sliced_proteins, _ = protein_to_slices(protein, target, max_slice_length=max_slice_length, padding=padding)
    dataloader = data.DataLoader(sliced_proteins, batch_size=1, shuffle=False)
    with torch.no_grad():
        for batch in dataloader:
            batch = utils.cuda(batch, device=torch.device(f'cuda:{GPU}'))
            pred = pipeline.task.predict({"graph": batch})
            intermediate_preds.append(pred)
    final_preds = torch.zeros(target.shape)
    for i, (start, end) in enumerate(get_slices(target.shape[0], max_slice_length=max_slice_length, padding=padding)):
        final_preds[start:end] += intermediate_preds[i].cpu()
        if i > 0:
            final_preds[start:start+padding] /= 2
    return final_preds



def get_pred_and_target_with_sliced_dataset(pipeline, data_set):
    preds = []
    targets = []
    for item in data_set:
        pred = infer_sliced(pipeline, item['graph'])
        preds.append(pred)

        dataloader = data.DataLoader([item], batch_size=1, shuffle=False)    
        target = pipeline.task.target(next(iter(dataloader)))
        targets.append(target)
        
    pred = utils.cat(preds)
    target = utils.cat(targets)
    return pred, target


pred, target = get_pred_and_target_with_sliced_dataset(pipeline, test_set)

print(pred)

metric = pipeline.task.evaluate(pred, target, threshold=-2.9)
round_dict(dict_tensor_to_num(metric), 4)

tensor([[ -8.8587],
        [ -7.7427],
        [ -9.4689],
        ...,
        [-14.3080],
        [-13.4456],
        [-13.2479]])


{'sensitivity': 0.635,
 'precision': 0.6926,
 'mcc': 0.6479,
 'micro_auprc': 0.6661}

In [4]:
metric = pipeline.task.evaluate(pred, target, threshold=-3)
round_dict(dict_tensor_to_num(metric), 4)

{'sensitivity': 0.635,
 'precision': 0.6903,
 'mcc': 0.6467,
 'micro_auprc': 0.6661}

In [8]:
pipeline.valid_dataset_stats()

{'best_mcc': 0.6806598102694253,
 'best_threshold': -1.0999999999999999,
 'loss': 0.16789284483952957}

In [6]:
def get_best_mcc(pipeline, valid_set):
    pred, target = get_pred_and_target_with_sliced_dataset(pipeline, valid_set)
    thresholds = np.linspace(-3, 1, num=41)
    mcc_values = [pipeline.task.evaluate(pred, target, threshold=th)['mcc'] for th in thresholds]
    
    max_mcc_idx = np.argmax(mcc_values)
    print(mcc_values)
    return round(thresholds[max_mcc_idx], 2), mcc_values[max_mcc_idx]


best_threshold, best_mcc = get_best_mcc(pipeline, valid_set)
print(best_threshold, best_mcc)

[0.6719496756019859, 0.6737841390670234, 0.6739642661708714, 0.6745887111112963, 0.6753388171835185, 0.6746966688503381, 0.6750844733485658, 0.6757992916735809, 0.6759438984696856, 0.6761720045849945, 0.6752558695999953, 0.6749297830832559, 0.6760232566910069, 0.6761153466905574, 0.6769829930959705, 0.6772850143018433, 0.6773653668131454, 0.6780504060088259, 0.6795738626874842, 0.6806598102694253, 0.6788067006387218, 0.6788478563459069, 0.6783976359865879, 0.6790427861905656, 0.6787444424793512, 0.6778632103042203, 0.6772055924489395, 0.6784831411937146, 0.6764918169585021, 0.6768154760072012, 0.6771480966802296, 0.6755352387470382, 0.6754955418239035, 0.6748646138546883, 0.6734310253912156, 0.6710088450901409, 0.6702001563890424, 0.6685808301139903, 0.667958744321504, 0.6669304098434377, 0.6681347185682568]
-1.1 0.6806598102694253


In [9]:
metric = pipeline.task.evaluate(pred, target, threshold=best_threshold)
round_dict(dict_tensor_to_num(metric), 4)

{'sensitivity': 0.5964,
 'precision': 0.7585,
 'mcc': 0.6591,
 'micro_auprc': 0.6661}

In [10]:
pipeline.evaluate_test_sliced(best_threshold)

evaluate test sliced with threshold: -1.1, max_slice_length: 650, padding: 50


{'sensitivity': 0.5964391827583313,
 'precision': 0.7584905624389648,
 'mcc': 0.6590645086899897,
 'micro_auprc': 0.6661270260810852}

In [15]:
import pandas as pd

df = pd.read_csv('result_cv/result_cv.csv')

In [13]:
df.groupby(['max_slice_length', 'padding']).agg({'mcc': ['mean', 'std', 'count']}).sort_values(('mcc', 'mean'), ascending=False)

mcc                
                             mean       std count
max_slice_length padding                         
500              50       0.66661  0.012100    10
                 100      0.66616  0.011255    10
600              50       0.66508  0.017986    10
500              150      0.66318  0.012937    10
400              150      0.66289  0.013234    10
600              100      0.66254  0.013314    10
                 150      0.66251  0.014203    10
400              50       0.66194  0.009623    10
                 100      0.66124  0.008999    10
450              50       0.65470       NaN     1
                 100      0.64890       NaN     1
550              100      0.64340       NaN     1
                 50       0.64120       NaN     1

In [10]:
df.groupby(['max_slice_length']).agg({'micro_auprc': ['mean', 'std', 'count']}).sort_values(('micro_auprc', 'mean'), ascending=False)

micro_auprc                
                        mean       std count
max_slice_length                            
500                 0.679240  0.014041    30
600                 0.677303  0.018748    30
400                 0.672220  0.014127    30

In [5]:
df_gearnet = df.query('model_key == "esm-33-gearnet"').groupby(['model_key', 'max_slice_length', 'padding']).agg({'mcc': ['mean', 'std'], 'micro_auprc': ['mean', 'std']})

df_gearnet.sort_values(('mcc', 'mean'), ascending=False)

mcc           micro_auprc  \
                                            mean       std        mean   
model_key      max_slice_length padding                                  
esm-33-gearnet 600              50       0.67416  0.019032     0.68404   
               500              50       0.67294  0.005925     0.68840   
                                100      0.67108  0.008269     0.69126   
               600              100      0.66780  0.009574     0.67560   
               400              150      0.66732  0.015231     0.68038   
               600              150      0.66644  0.014063     0.67778   
               400              100      0.66420  0.002512     0.67318   
               500              150      0.66394  0.007261     0.67286   
               400              50       0.66130  0.012414     0.66714   

                                                   
                                              std  
model_key      max_slice_length padding            
esm-33-gearnet 600              50       0.013791  
               500              50       0.015422  
                                100      0.006638  
               600              100      0.015364  
               400              150      0.019357  
               600              150      0.023404  
               400              100      0.011562  
               500              150      0.006226  
               400              50       0.011698

In [6]:
df_gearnet = df.query('model_key == "esm-33-gearnet"').groupby(['model_key', 'max_slice_length', 'padding']).agg({'mcc': ['mean', 'std'], 'micro_auprc': ['mean', 'std']})

df_gearnet.sort_values(('micro_auprc', 'mean'), ascending=False)

mcc           micro_auprc  \
                                            mean       std        mean   
model_key      max_slice_length padding                                  
esm-33-gearnet 500              100      0.67108  0.008269     0.69126   
                                50       0.67294  0.005925     0.68840   
               600              50       0.67416  0.019032     0.68404   
               400              150      0.66732  0.015231     0.68038   
               600              150      0.66644  0.014063     0.67778   
                                100      0.66780  0.009574     0.67560   
               400              100      0.66420  0.002512     0.67318   
               500              150      0.66394  0.007261     0.67286   
               400              50       0.66130  0.012414     0.66714   

                                                   
                                              std  
model_key      max_slice_length padding            
esm-33-gearnet 500              100      0.006638  
                                50       0.015422  
               600              50       0.013791  
               400              150      0.019357  
               600              150      0.023404  
                                100      0.015364  
               400              100      0.011562  
               500              150      0.006226  
               400              50       0.011698

In [11]:
df.query('model_key == "esm-33-gearnet" and max_slice_length == 500 and padding == 50')

,model_key,valid_fold,max_slice_length,padding,sensitivity,precision,mcc,micro_auprc,valid_mcc,train_bce,valid_bce,lr,finished_at
46,esm-33-gearnet,0,500,50,0.6306,0.7456,0.6721,0.6812,0.6919,0.0005,0.1655,0.0003,2024-06-06 06:40:15
55,esm-33-gearnet,1,500,50,0.6142,0.7753,0.6772,0.7011,0.6036,0.0007,0.2160,0.0003,2024-06-06 08:14:07
64,esm-33-gearnet,2,500,50,0.6409,0.7500,0.6800,0.7050,0.6150,0.0011,0.1787,0.0003,2024-06-06 09:48:00
73,esm-33-gearnet,3,500,50,0.6098,0.7668,0.6706,0.6669,0.6953,0.0006,0.1498,0.0003,2024-06-06 11:21:40
82,esm-33-gearnet,4,500,50,0.6157,0.7477,0.6648,0.6878,0.6525,0.0007,0.1572,0.0003,2024-06-06 12:55:18


In [17]:
df.query('max_slice_length == 450')

,model_key,valid_fold,max_slice_length,padding,sensitivity,precision,mcc,micro_auprc,valid_mcc,train_bce,valid_bce,lr,finished_at


In [18]:
df.query('model_key == "esm-t33"').groupby(['model_key', 'max_slice_length', 'padding']).agg({'mcc': ['mean', 'std'], 'micro_auprc': ['mean', 'std']}).sort_values(('micro_auprc', 'mean'), ascending=False)

mcc           micro_auprc          
                                       mean       std        mean       std
model_key max_slice_length padding                                         
esm-t33   500              100      0.66124  0.012495     0.68172  0.010331
          600              50       0.65600  0.012630     0.67948  0.013372
                           100      0.65728  0.015427     0.67362  0.026564
                           150      0.65858  0.014749     0.67330  0.024212
          400              150      0.65846  0.010629     0.67328  0.013004
                           50       0.66258  0.007294     0.67244  0.011791
          450              100      0.64890       NaN     0.67230       NaN
          500              50       0.66028  0.013934     0.67080  0.013911
                           150      0.66242  0.017956     0.67040  0.016663
          400              100      0.65828  0.012410     0.66690  0.018439
          550              100      0.64340       NaN     0.66370       NaN
          450              50       0.65470       NaN     0.65360       NaN
          550              50       0.64120       NaN     0.65210       NaN

In [23]:
df.query('max_slice_length == 500 and padding == 50 and model_key == "esm-t33"')

,model_key,valid_fold,max_slice_length,padding,sensitivity,precision,mcc,micro_auprc,valid_mcc,train_bce,valid_bce,lr,finished_at
1,esm-t33,0,500,50,0.6202,0.6990,0.6432,0.6493,0.6694,0.0008,0.1976,0.0003,2024-06-06 02:08:59
10,esm-t33,1,500,50,0.6484,0.7211,0.6696,0.6749,0.5999,0.0018,0.2138,0.0003,2024-06-06 03:05:27
19,esm-t33,2,500,50,0.6217,0.7456,0.6671,0.6843,0.5673,0.0013,0.1922,0.0003,2024-06-06 03:56:52
28,esm-t33,3,500,50,0.6231,0.7581,0.6740,0.6800,0.6814,0.0014,0.1636,0.0003,2024-06-06 04:48:18
37,esm-t33,4,500,50,0.5786,0.7558,0.6475,0.6655,0.6386,0.0012,0.1873,0.0003,2024-06-06 05:39:36
